# CAR DAMAGE DETECTION 
### TEAM NEXUS 



## Step 1 - training data location

S3 bucket containing the image dataset. 

```
images_to_classify
├── defect_free
│   ├── 1.jpg
│   ├── 2.jpg
|   ├── 3.jpg
│   └── . . .
└── defective
│   ├── 1.jpg
│   ├── 2.jpg    
│   ├── 3.jpg
│   ├── . . .
└── . . .
```

In [1]:
pip install sagemaker==1.72.0

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 KB 8.0 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 52.4 MB/s eta 0:00:00:00:0100:01
  Created wheel for sagemaker: filename=sagemaker-1.72.0-py2.py3-none-any.whl size=386379 sha256=8b4d9a585dbab789a80528a3e3a764f53b46911a13f43d83ffe1b97914853e88
  Stored in directory: /home/ec2-user/.cache/pip/wheels/86/0e/1f/7c255f46c88da77dc4d0145188d39efda49e7e69fa9d320edf
Successfully built sagemaker
  Attempting uninstall: smdebug-rulesconfig
    Found existing installation: smdebug-rulesconfig 1.0.1
    Uninstalling smdebug-rulesconfig-1.0.1:
      Successfully uninstalled smdebug-rulesconfig-1.0.1
  Attempting uninstall: botocore
    Found existing installation: botocore 1.24.19
    Uninstalling botocore-1.24.19:
      Successfully uninstalled botocore-1.24.19
  Attempting uninstall

In [2]:
pip install mxnet

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 40.5 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:


# The S3 bucket name where the image dataset is located
data_bucket_name='mynovpotatobucket'

# The name of the folder containing the dataset
dataset_name = 'data1a'



## Step 2 - Setup environment

The following imports the neccesary python libraries and fetches the execution role for the notebook instance. It also fetches the built-in image-classification algorithm.

In [4]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

role = get_execution_role()
sess = sagemaker.Session()

training_image = get_image_uri(sess.boto_region_name, 'image-classification', repo_version="latest")

# Find im2rec in our environment and set up some other vars in our environemnt

base_dir='/tmp'

%env BASE_DIR=$base_dir
%env S3_DATA_BUCKET_NAME = $data_bucket_name
%env DATASET_NAME = $dataset_name

import sys,os

suffix='/mxnet/tools/im2rec.py'
im2rec = list(filter( (lambda x: os.path.isfile(x + suffix )), sys.path))[0] + suffix
%env IM2REC=$im2rec


'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


env: BASE_DIR=/tmp
env: S3_DATA_BUCKET_NAME=mynovpotatobucket
env: DATASET_NAME=data1a
env: IM2REC=/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/mxnet/tools/im2rec.py


## Step 3 - Download image dataset from Amazon S3

In [5]:
# Pull our images from S3
!aws s3 sync s3://$S3_DATA_BUCKET_NAME/$DATASET_NAME $BASE_DIR/$DATASET_NAME 

download: s3://mynovpotatobucket/data1a/training/00-damage/0001.JPEG to ../../../tmp/data1a/training/00-damage/0001.JPEG
download: s3://mynovpotatobucket/data1a/training/00-damage/0003.JPEG to ../../../tmp/data1a/training/00-damage/0003.JPEG
download: s3://mynovpotatobucket/data1a/training/00-damage/0002.JPEG to ../../../tmp/data1a/training/00-damage/0002.JPEG
download: s3://mynovpotatobucket/data1a/training/00-damage/0004.JPEG to ../../../tmp/data1a/training/00-damage/0004.JPEG
download: s3://mynovpotatobucket/data1a/training/00-damage/0007.JPEG to ../../../tmp/data1a/training/00-damage/0007.JPEG
download: s3://mynovpotatobucket/data1a/training/00-damage/0010.JPEG to ../../../tmp/data1a/training/00-damage/0010.JPEG
download: s3://mynovpotatobucket/data1a/training/00-damage/0005.JPEG to ../../../tmp/data1a/training/00-damage/0005.JPEG
download: s3://mynovpotatobucket/data1a/training/00-damage/0013.JPEG to ../../../tmp/data1a/training/00-damage/0013.JPEG
download: s3://mynovpotatobucket

## Step 4 - Create RecordIO files from images

The images need to be converted into RecordIO files. Using im2rec.py, These files contain binary data of the images indexed by class, which is infered by the folder structure. Two files are made for training and validation. 

For more information see https://docs.aws.amazon.com/sagemaker/latest/dg/image-classification.html#IC-inputoutput

In [6]:
%%bash
# Use the IM2REC script to convert our images into RecordIO files

# Clean up our working dir of existing LST and REC files
cd $BASE_DIR
rm *.rec
rm *.lst

# First we need to create two LST files (training and test lists), noting the correct label class for each image
# We'll also save the output of the LST files command, since it includes a list of all of our label classes
echo "Creating LST files"
python $IM2REC --list --recursive --pass-through --test-ratio=0.3 --train-ratio=0.7 $DATASET_NAME $DATASET_NAME > ${DATASET_NAME}_classes

echo "Label classes:"
cat ${DATASET_NAME}_classes

# Then we create RecordIO files from the LST files
echo "Creating RecordIO files"
python $IM2REC --num-thread=4 ${DATASET_NAME}_train.lst $DATASET_NAME
python $IM2REC --num-thread=4 ${DATASET_NAME}_test.lst $DATASET_NAME
ls -lh *.rec

Creating LST files
Label classes:
training/00-damage 0
training/01-whole 1
validation/00-damage 2
validation/01-whole 3
Creating RecordIO files
Creating .rec file from /tmp/data1a_train.lst in /tmp
time: 0.5532736778259277  count: 0
time: 6.650928497314453  count: 1000
Creating .rec file from /tmp/data1a_test.lst in /tmp
time: 0.0040018558502197266  count: 0
-rw-rw-r-- 1 ec2-user ec2-user  45M Nov 26 15:31 data1a_test.rec
-rw-rw-r-- 1 ec2-user ec2-user 110M Nov 26 15:31 data1a_train.rec


rm: cannot remove ‘*.rec’: No such file or directory
rm: cannot remove ‘*.lst’: No such file or directory


## Step 5 - Upload RecordIO files to S3

In [7]:
# Upload our train and test RecordIO files to S3 in the bucket that our sagemaker session is using
bucket = sess.default_bucket()

s3train_path = 's3://{}/{}/train/'.format(bucket, dataset_name)
s3validation_path = 's3://{}/{}/validation/'.format(bucket, dataset_name)

# Clean up any existing data
!aws s3 rm s3://{bucket}/{dataset_name}/train --recursive
!aws s3 rm s3://{bucket}/{dataset_name}/validation --recursive

# Upload the rec files to the train and validation channels
!aws s3 cp /tmp/{dataset_name}_train.rec $s3train_path
!aws s3 cp /tmp/{dataset_name}_test.rec $s3validation_path

upload: ../../../tmp/data1a_train.rec to s3://sagemaker-us-east-1-606994673930/data1a/train/data1a_train.rec
upload: ../../../tmp/data1a_test.rec to s3://sagemaker-us-east-1-606994673930/data1a/validation/data1a_test.rec


## Step 6 - Configure training model



In [10]:
# The minimum batch size should be less than the total sample size for each class. 
batch_size = 2

train_data = sagemaker.session.s3_input(
    s3train_path, 
    distribution='FullyReplicated', 
    content_type='application/x-recordio', 
    s3_data_type='S3Prefix'
)

validation_data = sagemaker.session.s3_input(
    s3validation_path, 
    distribution='FullyReplicated', 
    content_type='application/x-recordio', 
    s3_data_type='S3Prefix'
)

data_channels = {'train': train_data, 'validation': validation_data}

s3_output_location = 's3://{}/{}/output'.format(bucket, dataset_name)

image_classifier = sagemaker.estimator.Estimator(
    training_image,
    role, 
    train_instance_count=1, 
    train_instance_type='ml.p3.2xlarge',
    output_path=s3_output_location,
    sagemaker_session=sess
)

num_classes=! ls -l {base_dir}/{dataset_name} | wc -l
num_classes=int(num_classes[0]) - 1

num_training_samples=! cat {base_dir}/{dataset_name}_train.lst | wc -l
num_training_samples = int(num_training_samples[0])



# These hyperparameters we won't want to change, as they define things like
# the size of the images we'll be sending for input, the number of training classes we have, etc.
base_hyperparameters=dict(
    use_pretrained_model=1,
    image_shape='3,50,100',
    num_classes=num_classes,
    num_training_samples=num_training_samples,
)

# These are hyperparameters we may want to tune, as they can affect the model training success:
hyperparameters={
    **base_hyperparameters, 
    **dict(
        learning_rate=0.001,
        mini_batch_size= batch_size,
    )
}


image_classifier.set_hyperparameters(**hyperparameters)

hyperparameters



's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


{'use_pretrained_model': 1,
 'image_shape': '3,50,100',
 'num_classes': 2,
 'num_training_samples': 1607,
 'learning_rate': 0.001,
 'mini_batch_size': 2}

## Step 7 - Start training job

This will dispatch a job to Amazon SageMaker to begin a training job. Upon completion it will upload a training model to Amazon S3 in a generate bucket for SageMaker.

In [11]:
%%time

import time
now = str(int(time.time()))
training_job_name = 'IC-' + dataset_name.replace('_', '-') + '-' + now

image_classifier.fit(inputs=data_channels, job_name=training_job_name, logs=True)

job = image_classifier.latest_training_job
model_path = f"{base_dir}/{job.name}"

print(f"\n\n Finished training! The model is available for download at: {image_classifier.output_path}/{job.name}/output/model.tar.gz")

2022-11-26 15:43:37 Starting - Starting the training job...
2022-11-26 15:44:04 Starting - Preparing the instances for training.........
2022-11-26 15:45:33 Downloading - Downloading input data
2022-11-26 15:45:33 Training - Downloading the training image..................
2022-11-26 15:48:29 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
[11/26/2022 15:48:35 INFO 140333708179264] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/image_classification/default-input.json: {'use_pretrained_model': 0, 'num_layers': 152, 'epochs': 30, 'learning_rate': 0.1, 'lr_scheduler_factor': 0.1, 'optimizer': 'sgd', 'momentum': 0, 'weight_decay': 0.0001, 'beta_1': 0.9, 'beta_2': 0.999, 'eps': 1e-08, 'gamma': 0.9, 'mini_batch_size': 32, 'image_shape': '3,224,224', 'precision_dtype': 'float32'}
[11/26/2022 15:48:35 INFO 140333708179264] Merging with provided configuration from /opt/ml/input/config/hyperparameters

## Step 8 - Deploy Model Endpoint

Deploys a REST endpoint that can be invoked in order to run images against for classification.

In [12]:


%%time
# Deploying a model to an endpoint takes a few minutes to complete

deployed_endpoint = image_classifier.deploy(
    initial_instance_count = 1,
    instance_type = 'ml.t2.medium'
)



Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


-------------------------!CPU times: user 339 ms, sys: 25.7 ms, total: 365 ms
Wall time: 12min 33s


In [16]:
import boto3, sys
sm_rt = boto3.Session().client('sagemaker-runtime')

In [23]:
import json

In [25]:
#response = sm_rt.invoke_endpoint(EndpointName='IC-data1a-1669477417', ContentType="application/json", Body=json.dumps("a json string"))
sm_boto3 = boto3.client("sagemaker")
ep_des_res = sm_boto3.describe_endpoint(EndpointName='IC-data1a-1669477417')
ep_des_res

{'EndpointName': 'IC-data1a-1669477417',
 'EndpointArn': 'arn:aws:sagemaker:us-east-1:606994673930:endpoint/ic-data1a-1669477417',
 'EndpointConfigName': 'IC-data1a-1669477417',
 'ProductionVariants': [{'VariantName': 'AllTraffic',
   'DeployedImages': [{'SpecifiedImage': '811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification:latest',
     'ResolvedImage': '811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification@sha256:3e050c7dd64e609774da1445deee0ef8c40e15ef1c0299b36b1259d34c4f744b',
     'ResolutionTime': datetime.datetime(2022, 11, 26, 16, 35, 26, 12000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 1,
   'DesiredInstanceCount': 1}],
 'EndpointStatus': 'InService',
 'CreationTime': datetime.datetime(2022, 11, 26, 16, 35, 25, 411000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 11, 26, 16, 47, 33, 939000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': 'e9b2f6cd-765f-4b74-beb7-9d8231816